## eoAPI in Action - Afghanistan Earthquakes 2022

The Jupyter Notebook is dedicated to analyzing the aftermath of the Afghanistan earthquakes in 2022.  The notebook should serve as an example for diving deeper into the available imagery pre and post event and assessing what it could be used for. 

Firstly, we'll delve into querying MAXAR's catalog of images of the event, and pinpointing those relevant to the regions affected by the earthquakes. 

If possible based on the images obtained, we'll shift our focus to finding signs of impact, employing sophisticated image analysis techniques to identify and interpret the changes in the landscape. Then we will try to determine the population affected by these seismic events, integrating demographic data with our impact analysis to estimate the extent of the humanitarian challenge. 

## Setup

Before we dive into the analysis, let's set up our environment by importing the necessary libraries. This step is crucial for ensuring that we have all the tools required for our data processing, analysis, and visualization tasks. 

In [41]:
import IPython
!python -m pip install httpx ipyleaflet matplotlib
IPython.display.clear_output(wait=False)


In [42]:
import json
from datetime import datetime
from dateutil.parser import parse

import httpx
import ipyleaflet
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

Defining some key environment variables upfront is essential to interact with the eoAPI and streamline our analysis effectively. These variables will be the foundational parameters for our API queries and data handling. The crucial variables we'll set include:

- stac_endpoint: This is the URL of the STAC (SpatioTemporal Asset Catalog) endpoint of eoAPI. It serves as the entry point for querying and retrieving metadata about the satellite imagery available in our catalog.

- collection_id: Each collection of images in the eoAPI is identified by a unique ID. By specifying our collection_id, we can access the specific dataset relevant to the Afghanistan earthquake regions.

- raster_endpoint: This variable defines the endpoint for accessing raster data within the eoAPI. It allows us to retrieve and manipulate satellite imagery, which is crucial for our impact analysis and visualization tasks.

In [43]:
stac_endpoint = "https://eoapi.ifrc-risk.k8s.labs.ds.io/stac"
collection_id = "MAXAR_afghanistan_earthquake22"
raster_endpoint = "https://eoapi.ifrc-risk.k8s.labs.ds.io/raster"
event_date_str =  "Wednesday, June 22, 2022"
event_datetime = datetime.strptime(event_date_str, "%A, %B %d, %Y")

## Querying eoAPI to Find Images

In this section, we will take the first critical step of our analysis, which is querying our image catalog using the eoAPI. This process involves accessing the vast repository of satellite data specific to the regions affected by the Afghanistan earthquakes. Our goal is to use the stac_endpoint and collection_id to filter and retrieve the images that are most relevant to our study. We want to streamline the querying process to efficiently select images that provide valuable insights into the seismic impacts. This step is not just about fetching data; it's about strategically selecting the right datasets to form the foundation of our subsequent analyses.

In [44]:

collection_info = httpx.get(f"{stac_endpoint}/collections/{collection_id}").json()
print(f"{collection_info['title'] if 'title' in collection_info else 'NA'}: {collection_info['description']}")

Afghanistan Earthquake: Maxar OpenData | On Wednesday, June 22, 2022, a 5.9 magnitude earthquake struck eastern Afghanistan. It has been estimated that more than 1,000 have been killed and many more wounded. Houses have been reduced to rubble and an unknown number of people remain stuck under debris. In outlaying areas rescue operations were complicated by difficult conditions including heavy rain, landslides and villages being located in inaccessible hillsides areas. Wednesday's earthquake was the deadliest in Afghanistan since 2022. It struck about 27 miles (44 km) from the southeastern city of Khost, near the border with Pakistan.


In [45]:
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [bbox[0], bbox[1]],
                    [bbox[2], bbox[1]],
                    [bbox[2], bbox[3]],
                    [bbox[0], bbox[3]],
                    [bbox[0], bbox[1]],
                ]]
            },
            'properties': {}
        }
        for bbox in collection_info["extent"]["spatial"]["bbox"] if "extent" in collection_info
    ]
}

mainbbox = collection_info["extent"]["spatial"]["bbox"][0]

m = ipyleaflet.leaflet.Map(
    center=((mainbbox[1] + mainbbox[3]) / 2,(mainbbox[0] + mainbbox[2]) / 2),
    zoom=5
)

geo_json = ipyleaflet.leaflet.GeoJSON(data=geojson)
m.add_layer(geo_json)
m

Map(center=[29.60082468671302, 79.33739695318904], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [46]:
# checking that the temporal extents are actually all the same!
temporal_extents = collection_info["extent"]["temporal"]["interval"]
temporal_extent = temporal_extents[0]
score = 0
for ext in temporal_extents:
    if ext != temporal_extent:
        print(ext)
        score += 1
score

0

In [47]:
items_url = f"{stac_endpoint}/collections/{collection_id}/items"
items = httpx.get(items_url).json() 
len(items)

4

In [48]:
afg_items = []

url = items_url
while True:
    items = httpx.get(url, params={"limit": 200}).json()
    
    afg_items.extend(items["features"])
    next_link = list(filter(lambda link: link["rel"] == "next", items["links"]))
    if next_link:
        url = next_link[0]["href"]
    else:
        break

print(f"Actual Number of Items: {len(afg_items)}")

Actual Number of Items: 395


In [49]:
afg_items_pre = []
afg_items_post = []

for item in afg_items:
    item_datetime = datetime.strptime(item['properties']['datetime'].replace('Z', ''), "%Y-%m-%dT%H:%M:%S") 
    if item_datetime < event_datetime:
        afg_items_pre.append(item_datetime)
    else:
        afg_items_post.append(item_datetime)

print(f"Number of items before event: {len(afg_items_pre)}")
print(f"Number of items after event: {len(afg_items_post)}")

Number of items before event: 223
Number of items after event: 172


Upon closer examination of our dataset, we have encountered several significant issues that must be addressed. Firstly, the temporal extents of the data we've queried are not well-formatted, which poses a challenge in accurately assessing the satellite capture timeline. This formatting issue could lead to difficulties in understanding the precise timeframes of the images, particularly in relation to the earthquake events. 

### Mosaicing the images

Even without specific post-event images, we will create a mosaic to simultaneously visualize all the pre-events. The mosaic could be overlaid with any custom data available from other sources.

In [50]:
item = items["features"][0]
tilejson = httpx.get(
    f"{raster_endpoint}/collections/{collection_id}/items/{item['id']}/tilejson.json",
    params = (
        ("assets", "visual"),  # THIS PARAMETER IS MANDATORY
        ("minzoom", 12),  # By default the tiler will use 0
        ("maxzoom", 19), # By default the tiler will use 24
    )
).json()
tilejson

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['http://eoapi.ifrc-risk.k8s.labs.ds.io/raster/collections/MAXAR_afghanistan_earthquake22/items/42_120200200330_10300100C084CC00/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=visual'],
 'minzoom': 12,
 'maxzoom': 19,
 'bounds': [69.31926076847,
  32.941313704603466,
  69.37629464192838,
  32.98939210855889],
 'center': [69.34777770519919, 32.965352906581174, 12]}

In [51]:
bounds = tilejson["bounds"]
m = ipyleaflet.leaflet.Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=12
)

geo_json = ipyleaflet.leaflet.GeoJSON(
    data=item,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0., 'weight': 4
    }
)
m.add_layer(geo_json)

tiles = ipyleaflet.leaflet.TileLayer(
    url=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],

    ],
)

m.add_layer(tiles)

m

Map(center=[32.965352906581174, 69.34777770519919], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [52]:
afg_items[0]

{'id': '42_120200201121_10300100D4928800',
 'bbox': [69.69514012966512,
  33.12480123241977,
  69.74715134156378,
  33.16826382393089],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://eoapi.ifrc-risk.k8s.labs.ds.io/stac/collections/MAXAR_afghanistan_earthquake22'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://eoapi.ifrc-risk.k8s.labs.ds.io/stac/collections/MAXAR_afghanistan_earthquake22'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://eoapi.ifrc-risk.k8s.labs.ds.io/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://eoapi.ifrc-risk.k8s.labs.ds.io/stac/collections/MAXAR_afghanistan_earthquake22/items/42_120200201121_10300100D4928800'}],
 'assets': {'visual': {'href': 's3://maxar-opendata/events/afghanistan-earthquake22/ard/42/120200201121/2022-06-27/10300100D4928800-visual.tif',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
   'r

In [55]:
event_date = "2023-02-06T00:00:00Z"

# Let's focus on Kahramanmaraş city 
bounds = [69.31926076847,
  32.941313704603466,
  69.37629464192838,
  32.98939210855889]

pre_mosaic = httpx.post(
    f"{raster_endpoint}/searches/register",
    data=json.dumps(
        {
            "filter-lang": 'cql2-json',
            "filter": {
                "op": 'and', 
                "args": [
                    {
                        "op": "in", 
                        "args": [{"property": "collection"}, [collection_id]]
                    },
                    {
                        "op": "lt", 
                        "args": [
                            {"property": "datetime"}, event_date
                        ]
                    }
                ],
            },
            "sortby": [
                {
                    "field": "tile:clouds_percent",
                    "direction": "asc"
                },
            ],
            "metadata":{
                "name": "Maxar Kahramanmaras - Pre event",
                "bounds": bounds,
            }
            
        }
    )
).json()
pre_mosaic

{'id': '34b016b66d4d5d0c1951d57bc85f7be3',
 'links': [{'rel': 'metadata',
   'title': 'Mosaic metadata',
   'type': 'application/json',
   'href': 'http://eoapi.ifrc-risk.k8s.labs.ds.io/raster/searches/34b016b66d4d5d0c1951d57bc85f7be3/info'},
  {'rel': 'tilejson',
   'title': 'Link for TileJSON',
   'type': 'application/json',
   'href': 'http://eoapi.ifrc-risk.k8s.labs.ds.io/raster/searches/34b016b66d4d5d0c1951d57bc85f7be3/tilejson.json'},
  {'rel': 'map',
   'title': 'Link for Map viewer',
   'type': 'application/json',
   'href': 'http://eoapi.ifrc-risk.k8s.labs.ds.io/raster/searches/34b016b66d4d5d0c1951d57bc85f7be3/map'},
  {'rel': 'wmts',
   'title': 'Link for WMTS',
   'type': 'application/json',
   'href': 'http://eoapi.ifrc-risk.k8s.labs.ds.io/raster/searches/34b016b66d4d5d0c1951d57bc85f7be3/WMTSCapabilities.xml'}]}

In [56]:
mosaic_id = pre_mosaic["id"]

tilejson_pre = httpx.get(
    f"{raster_endpoint}/searches/{mosaic_id}/tilejson.json",
    params = (
        ("assets", "visual"),  # required parameter
        ("minzoom", 12),
        ("maxzoom", 19), 
    )
).json()

bounds = tilejson_pre["bounds"]
m = ipyleaflet.leaflet.Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=12
)

geo_json = ipyleaflet.leaflet.GeoJSON(
    data={"type": "FeatureCollection", "features": pre_items}, 
    style={
        "fillOpacity": 0,
        "weight": 1,
    },
)
m.add_layer(geo_json)

tiles = ipyleaflet.leaflet.TileLayer(
    url=tilejson_pre["tiles"][0],
    min_zoom=tilejson_pre["minzoom"],
    max_zoom=tilejson_pre["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],

    ],
)

m.add_layer(tiles)
m

Map(center=[32.965352906581174, 69.34777770519919], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [57]:
event_date = datetime(2022, 6, 22, hour=0, minute=0).isoformat()

post_mosaic = httpx.post(
    f"{raster_endpoint}/searches/register",
    data=json.dumps(
        {
            "filter-lang": 'cql2-json',
            "filter": {
                "op": 'and', 
                "args": [
                    {
                        "op": "in", 
                        "args": [{"property": "collection"}, [collection_id]]
                    },
                    {
                        "op": "ge", 
                        "args": [
                            {"property": "datetime"}, event_date
                        ]
                    }
                ],
            },
            "sortby": [
                {
                    "field": "tile:clouds_percent",
                    "direction": "asc"
                },
            ],
            "metadata":{
                "name": "Maxar Afg - Post event",
                "bounds": bounds,
            }
            
        }
    )
).json()

mosaic_id = post_mosaic["id"]

tilejson_post = httpx.get(
    f"{raster_endpoint}/searches/{mosaic_id}/tilejson.json",
    params = (
        ("assets", "visual"),  # THIS IS MANDATORY
        ("minzoom", 0),
        ("maxzoom", 30), 
    )
).json()

bounds = tilejson_pre["bounds"]
before_layer = ipyleaflet.leaflet.TileLayer(
    url=tilejson_pre["tiles"][0],
    min_zoom=tilejson_pre["minzoom"],
    max_zoom=tilejson_pre["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],
    ],
)

bounds = tilejson_post["bounds"]
after_layer = ipyleaflet.leaflet.TileLayer(
    url=tilejson_post["tiles"][0],
    min_zoom=tilejson_post["minzoom"],
    max_zoom=tilejson_post["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],
    ],
)

control = ipyleaflet.leaflet.SplitMapControl(left_layer=before_layer, right_layer=after_layer)
m.add_control(control)

m

Map(bottom=53011.0, center=[33.22949814144951, 69.41436767578126], controls=(ZoomControl(options=['position', …